In [16]:
import os
import sys
sys.path.insert(0, os.path.abspath('../../../'))

import numpy as np
import arrayfire as af
from matplotlib import pyplot as plt

from dg_maxwell import params
from dg_maxwell import msh_parser
from dg_maxwell import advection_2d
from dg_maxwell import global_variables as gvar
from dg_maxwell import utils
from dg_maxwell import wave_equation_2d
from dg_maxwell import isoparam
from dg_maxwell import lagrange
from dg_maxwell import advection_2d as a2d
from dg_maxwell import advection_2d_arbit_mesh as a2d_arbit_mesh

af.set_backend(params.backend)
af.set_device(params.device)

plt.rcParams['figure.figsize']     = 12, 7.5
plt.rcParams['lines.linewidth']    = 1.5
plt.rcParams['font.family']        = 'serif'
plt.rcParams['font.weight']        = 'bold'
plt.rcParams['font.size']          = 20  
plt.rcParams['font.sans-serif']    = 'serif'
plt.rcParams['text.usetex']        = True
plt.rcParams['axes.linewidth']     = 1.5
plt.rcParams['axes.titlesize']     = 'medium'
plt.rcParams['axes.labelsize']     = 'medium'

plt.rcParams['xtick.major.size']   = 8
plt.rcParams['xtick.minor.size']   = 4
plt.rcParams['xtick.major.pad']    = 8
plt.rcParams['xtick.minor.pad']    = 8
plt.rcParams['xtick.color']        = 'k'
plt.rcParams['xtick.labelsize']    = 'medium'
plt.rcParams['xtick.direction']    = 'in'    

plt.rcParams['ytick.major.size']   = 8
plt.rcParams['ytick.minor.size']   = 4
plt.rcParams['ytick.major.pad']    = 8
plt.rcParams['ytick.minor.pad']    = 8
plt.rcParams['ytick.color']        = 'k'
plt.rcParams['ytick.labelsize']    = 'medium'
plt.rcParams['ytick.direction']    = 'in'
plt.rcParams['text.usetex']        = True
plt.rcParams['text.latex.unicode'] = True

gmshTranslator: Ending
gmshTranslator: Ending


In [17]:
params.mesh_file = '../../read_and_plot_mesh/mesh/square_10_10.msh'
params.c_x = 1.
params.c_y = 0.

advec_var = gvar.advection_variables(params.N_LGL, params.N_quad,
                                     params.x_nodes, params.N_Elements,
                                     params.c, params.total_time, params.wave,
                                     params.c_x, params.c_y, params.courant,
                                     params.mesh_file, params.total_time_2d)

/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1193: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)
/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1220: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


gmshTranslator: Initializing...
gmshTranslator: Mesh has 441 nodes.
gmshTranslator: Mesh has 144 elements.
gmshTranslator: Processed 589 lines.
gmshTranslator: There are 1 physical groups available: 
gmshTranslator:      > 0
gmshTranslator: Parsing nodes
gmshTranslator: Parsing elements
gmshTranslator: No rules for elements... skipping elements.
gmshTranslator: Parsing nodes
gmshTranslator: No rules for nodes... skipping nodes.
gmshTranslator: Parsing elements
advection_variables __init__ completed


In [18]:
u = advec_var.u_e_ij

dx_dxi  = 0.1
dy_deta = 0.1

element_lf_flux = a2d_arbit_mesh.lf_flux_all_edges(u, advec_var)

Done


In [26]:
c_x = 1.
c_y = 1.

In [31]:
# LF flux analytical calculation
center_element_tag = 44
edge_id            = 0
u_center_element = af.moddims(advec_var.u_e_ij[:, element_tag],
                              d0 = params.N_LGL, d1 = params.N_LGL)

In [34]:
left_element_tag = 43
u_left_element = af.moddims(advec_var.u_e_ij[:, left_element_tag],
                            d0 = params.N_LGL, d1 = params.N_LGL)

lf_flux_ref_left_element = (c_x * center_element_u + c_x * u_left_element) / 2

In [41]:
print(af.moddims(advec_var.u_e_ij[:, 54], d0 = params.N_LGL, d1 = params.N_LGL))
print(a2d_arbit_mesh.u_at_edge(advec_var.y_e_ij[:, 54], 1))

arrayfire.Array()
Type: double

[8 8 1 1]
    0.6065     0.6257     0.6647     0.7108     0.7490     0.7707     0.7780     0.7788 
    0.6257     0.6454     0.6857     0.7332     0.7726     0.7950     0.8025     0.8034 
    0.6647     0.6857     0.7286     0.7790     0.8208     0.8447     0.8527     0.8536 
    0.7108     0.7332     0.7790     0.8329     0.8777     0.9032     0.9117     0.9127 
    0.7490     0.7726     0.8208     0.8777     0.9248     0.9517     0.9607     0.9617 
    0.7707     0.7950     0.8447     0.9032     0.9517     0.9794     0.9886     0.9896 
    0.7780     0.8025     0.8527     0.9117     0.9607     0.9886     0.9979     0.9990 
    0.7788     0.8034     0.8536     0.9127     0.9617     0.9896     0.9990     1.0000 


arrayfire.Array()
Type: double

[1 8 1 1]
    0.0000     0.0000     0.0000     0.0000     0.0000     0.0000     0.0000    -0.0000 

